In [7]:
# 1. Setup function tools for ReAct Agent
from langchain_core.tools import tool


@tool("find_sum", return_direct=True)
def find_sum(x:int, y:int) -> int:
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return the sum of two numbers.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y    

@tool("find_product", return_direct=True)
def find_product(x:int, y:int) -> int:
    """
    This function is used to multiple two numbers and return the product of two numbers.
    It takes two integers as inputs and returns an integer as output.
    """
    return x * y



In [10]:
# Create a basic ReAct Agent that can use the above tools to perform calculations
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv() ## loading the .env file

model = ChatOpenAI(model_name="gpt-4o")

In [ ]:
# Create a basic ReAct Agent that can use the above tools to perform calculations
from langchain_classic.agents import create_react_agent
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt: provides persona for the agent and instructions on how to use the tools
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available. 
    Do not solve the problem yourself"""
)

agent_graph = create_agent(model=model, tools=agent_tools, system_prompt=system_prompt)


In [19]:
## Execute the ReAct Agent with a user query
input_query = {"messages":[("user","what is the sum of 2 and 3 ?")]}

agent_response = agent_graph.invoke(input_query)
print("Agent returned:", agent_response['messages'][-1].content) ## print the final answer from the agent)

print("Step by step reasoning and tool usage:")
for message in agent_response['messages']:
    print(message.pretty_repr())
    
    

Agent returned: 5
Step by step reasoning and tool usage:
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_a0NPYQ6vXrmaa6yue9TCXMQg)
 Call ID: call_a0NPYQ6vXrmaa6yue9TCXMQg
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5


In [ ]:
#Example 2 with a complex prompt

inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : 6 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 1 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (call_lFjQSStZqtYKFAcoGqHFXoTC)
 Call ID: call_lFjQSStZqtYKFAcoGqHFXoTC
  Args:
    x: 3
    y: 2
  find_sum (call_rpZ0TEbEN7gb2zJV3UMJZhvA)
 Call ID: call_rpZ0TEbEN7gb2zJV3UMJZhvA
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6


In [25]:
# Debugging agents with LangGraph
agent_graph = create_agent(model=model, tools=agent_tools, system_prompt=system_prompt, debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

print("\n***********************\nDebugging the agent with LangGraph : \n")

for chunk in agent_graph.stream(inputs, stream_mode="debug"):
    print(chunk)


[values] {'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='e4502b82-0b2a-4337-9a5b-e05afc790e88')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 164, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_64dfa806c7', 'id': 'chatcmpl-D9z645FjCBhp64pjyNz7jJ7yrmqux', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c680c-90b9-7f90-b5d7-6c782c2ca6ee-0', tool_calls=[{'name': 'find_sum', 'args': {'x': 2, 'y': 3}, 'id': 'call_yNzf1EvaXd7nQHupDtLnDtkx', 'type': 'tool_call'}], invalid_tool_calls=[], usage_met